# Consolidate Output
This program reads in the data from runs on the BlueBEAR cluster via RDS and appends them on to the existing `malatium` and `copper` dataframes, saving them as `atium` and `bronze` respectively.

## What are we flagging?

We want to flag:
- Poor convergence in the splitting and inclination values
- Rhat values
- The effective n percentage
- The location on the HR diagram
 - MS: log(g) > 4.2 and Teff < 6250
 - SG: log(g) < 4.2
 - H: log(g) > 4.2 and Teff > 6250

In [1]:
import numpy as np
import lightkurve as lk
import pandas as pd
import fnmatch as fnm
import seaborn as sns
import astropy.units as u
from tqdm import tqdm
import glob
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)


Read in the existing dataframe

In [2]:
cols = ['KIC','Teff','age','dnu','eTeff','ednu','efeh','enumax','feh','loage','lomodlogg','lomodmass','lomodrad','modlogg','modmass','modrad','numax','source','upage','upmodlogg', 'upmodmass','upmodrad','G','bprp']
mal = pd.read_csv('../../data/malatium.csv', usecols=cols)
cop = pd.read_csv('../../data/copper.csv', index_col=0)

## Add the classification

In [3]:
mal['hrclass'] = ''
s = np.where((mal.modlogg > 4.2) & (mal.Teff < 6250))[0]
mal.loc[s, 'hrclass'] = 'MS'
s = np.where((mal.modlogg < 4.2))[0]
mal.loc[s, 'hrclass'] = 'SG'
s = np.where((mal.modlogg > 4.2) & (mal.Teff > 6250))[0]
mal.loc[s, 'hrclass'] = 'H'

## Add the mode results

Parse all the data, including some flags:

`empty   : run not yet done`

`rhat_1  : has at least one rhat > 1.1. Major concern, exclude from sample`

`nosum   : there is no summary available for some reason`


In [4]:
cop['f'] = np.nan
cop['f_e'] = np.nan
cop['g'] = np.nan
cop['g_e'] = np.nan
cop['A'] = np.nan
cop['A_e'] = np.nan
cop['H'] = np.nan
cop['H_e'] = np.nan
cop['flag'] = ''

In [5]:
incomplete = 0
for idx in tqdm(range(95)):
    kic = str(mal.loc[idx].KIC)
    files = glob.glob('/home/oliver/PhD/mnt/RDS/malatium/peakbag/{}/*chains.csv'.format(str(kic)))

    try:
        chains = pd.read_csv(files[0],index_col=0)
        lis = list(chains)
    except IndexError:
        incomplete += 1
        cop.loc[cop.KIC == kic, 'flag'] = 'empty'
        continue

    # Lets do each mode in turn
    for mode in [0,1,2]:
        cop.loc[(cop.KIC==kic)&(cop.l==mode),'f'] = np.median(chains[fnm.filter(lis, f'f{mode}_*')], axis=0)
        cop.loc[(cop.KIC==kic)&(cop.l==mode),'f_e'] = np.std(chains[fnm.filter(lis, f'f{mode}_*')], axis=0).values
        cop.loc[(cop.KIC==kic)&(cop.l==mode),'g'] = np.median(chains[fnm.filter(lis, f'g{mode}_*')], axis=0)
        cop.loc[(cop.KIC==kic)&(cop.l==mode),'g_e'] = np.std(chains[fnm.filter(lis, f'g{mode}_*')], axis=0).values
        cop.loc[(cop.KIC==kic)&(cop.l==mode),'A'] = np.median(chains[fnm.filter(lis, f'a{mode}_*')], axis=0)
        cop.loc[(cop.KIC==kic)&(cop.l==mode),'A_e'] = np.std(chains[fnm.filter(lis, f'a{mode}_*')], axis=0).values
        cop.loc[(cop.KIC==kic)&(cop.l==mode),'H'] = np.median(chains[fnm.filter(lis, f'h{mode}_*')], axis=0)
        cop.loc[(cop.KIC==kic)&(cop.l==mode),'H_e'] = np.std(chains[fnm.filter(lis, f'h{mode}_*')], axis=0).values    
    
    #Lets copy over the Rhats for each mode and flag if needed
    try:
        summary = pd.read_csv(files[0].replace('chains','summary'), index_col=0)
        rhats = summary.loc[fnm.filter(lis, '*__*')].Rhat.values
        
        if any(np.abs(rhats - 1.) >= 0.1):
            print('Rhats greater than 1.1 for KIC {}'.format(str(kic)))
            cop.loc[cop.KIC==kic, 'flag'] = 'rhat_1'

        
    except FileNotFoundError:
        print('No summary file for KIC {}'.format(str(kic)))
        mal.loc[idx, 'flag'] = 'nosum'
        pass
print(f'There are {incomplete} stars yet to run.')    

 74%|███████▎  | 70/95 [00:37<00:08,  2.80it/s]

Rhats greater than 1.1 for KIC 4143755


100%|██████████| 95/95 [00:46<00:00,  2.62it/s]

There are 5 stars yet to run.


In [6]:
cop.to_csv('../../data/bronze.csv')

### Add the parameters results

Read in the existing dataframes

In [7]:
#Add the necessary columns to malatium
mal['nus*'] = np.nan
mal['l_nus*'] = np.nan
mal['u_nus*'] = np.nan

mal['cosi'] = np.nan
mal['l_cosi'] = np.nan
mal['u_cosi'] = np.nan

mal['i'] = np.nan
mal['l_i'] = np.nan
mal['u_i'] = np.nan

mal['nus'] = np.nan
mal['l_nus'] = np.nan
mal['u_nus'] = np.nan

mal['P'] = np.nan
mal['l_P'] = np.nan
mal['u_P'] = np.nan

mal['vsini'] = np.nan
mal['l_vsini'] = np.nan
mal['u_vsini'] = np.nan

mal['fit_numax'] = np.nan
mal['efit_numax'] = np.nan
mal['fit_d01'] = np.nan
mal['efit_d01'] = np.nan
mal['fit_d02'] = np.nan
mal['efit_d02'] = np.nan
mal['fit_epsilon'] = np.nan
mal['efit_epsilon'] = np.nan

mal['V1'] = np.nan
mal['eV1'] = np.nan

mal['V2'] = np.nan
mal['eV2'] = np.nan

mal['rhat_flag'] = ''
mal['corner_flag'] = np.nan
mal['neff_xsplit'] = np.nan
mal['neff_cosi'] = np.nan
mal['neff_split'] = np.nan
mal['neff_i'] = np.nan

In [ ]:
incomplete = 0
for idx in tqdm(range(95)):
    kic = mal.loc[idx].KIC
    files = glob.glob('/home/oliver/PhD/mnt/RDS/malatium/peakbag/{}/*chains.csv'.format(str(kic)))

    # Check to see if run is done
    try:
        chains = pd.read_csv(files[0],index_col=0)
        niters = len(chains)
    except IndexError:
        incomplete += 1
        mal.loc[idx, 'rhat_flag'] = 'empty'
        continue

    #Flag Rhat values & Neff values
    try:
        summary = pd.read_csv(files[0].replace('chains','summary'), index_col=0)
        rhats = summary.loc[['xsplit','cosi','i','split']].Rhat.values
        
        if any(np.abs(rhats - 1.) >= 0.01):
            print('Rhats greater than 1.01 for KIC {}'.format(str(kic)))
            mal.loc[idx, 'rhat_flag'] = 'rhat_01'  
        
        if any(np.abs(rhats - 1.) >= 0.1):
            print('Rhats greater than 1.1 for KIC {}'.format(str(kic)))
            mal.loc[idx, 'rhat_flag'] = 'rhat_1'

        neff = summary.loc[['xsplit','cosi','i','split']].n_eff
        mal.loc[idx, 'neff_xsplit'] = int(neff['xsplit'])
        mal.loc[idx, 'neff_cosi'] = int(neff['cosi'])
        mal.loc[idx, 'neff_split'] = int(neff['split'])
        mal.loc[idx, 'neff_i'] = int(neff['i'])
            
    except FileNotFoundError:
        mal.loc[idx, 'flag'] = 'nosum'
        pass
    
    l, m, up = np.percentile(chains['xsplit'].values,[15.9, 50, 84.1])
    mal.loc[idx, 'l_nus*'], mal.loc[idx, 'nus*'], mal.loc[idx, 'u_nus*'] = m-l, m, up-m
    
    l, m, up = np.percentile(chains['cosi'].values, [15.9, 50, 84.1])
    mal.loc[idx, 'l_cosi'], mal.loc[idx, 'cosi'], mal.loc[idx, 'u_cosi'] = m-l, m, up-m
    
    l, m, up = np.percentile(chains['split'].values,[15.9, 50, 84.1])
    mal.loc[idx, 'l_nus'], mal.loc[idx, 'nus'], mal.loc[idx, 'u_nus'] = m-l, m, up-m
    
    l, m, up = np.percentile(chains['i'].values, [15.9, 50, 84.1])    
    mal.loc[idx, 'l_i'], mal.loc[idx, 'i'], mal.loc[idx, 'u_i'] =  m-l, m, up-m
    
    m, up = np.percentile(chains['numax'].values, [50, 84.1])    
    mal.loc[idx, 'fit_numax'], mal.loc[idx, 'efit_numax'] =  m, up-m

    m, up = np.percentile(chains['d01'].values, [50, 84.1])    
    mal.loc[idx, 'fit_d01'], mal.loc[idx, 'efit_d01'] =  m, up-m
    
    m, up = np.percentile(chains['d02'].values, [50, 84.1])    
    mal.loc[idx, 'fit_d02'], mal.loc[idx, 'efit_d02'] =  m, up-m
    
    m, up = np.percentile(chains['epsilon'].values, [50, 84.1])    
    mal.loc[idx, 'fit_epsilon'], mal.loc[idx, 'efit_epsilon'] =  m, up-m
    
    m, up = np.percentile(chains['V1'].values, [50, 84.1])    
    mal.loc[idx, 'V1'], mal.loc[idx, 'eV1'] =  m, up-m
    
    m, up = np.percentile(chains['V2'].values, [50, 84.1])    
    mal.loc[idx, 'V2'], mal.loc[idx, 'eV2'] =  m, up-m    
    
    nus = u.Quantity(chains['split'].values, u.microhertz)
    Pchain = 1./nus.to(1./u.day).value
    l, m, up = np.percentile(Pchain, [15.9, 50, 84.1])  
    mal.loc[idx, 'l_P'], mal.loc[idx, 'P'], mal.loc[idx, 'u_P'] = m-l, m, up-m
    
    Rkm = u.Quantity(mal.loc[idx, 'modrad'], u.solRad).to(u.km)
    vsinichain = np.sin(chains['i'].values) * 2 * np.pi * Rkm / (1./nus.to(u.Hertz).value)
    l, m, up = np.percentile(vsinichain, [15.9, 50, 84.1])  
    mal.loc[idx, 'l_vsini'], mal.loc[idx, 'vsini'], mal.loc[idx, 'u_vsini'] = m-l, m, up-m
    
print(f'There are {incomplete} stars still to run.') 
print(f"Of these, {len(np.where((mal.rhat_flag == 'empty') & (mal.hrclass == 'MS'))[0])} lie in the MS region.")

 14%|█▎        | 13/95 [00:05<00:40,  2.01it/s]

In [ ]:
mal.columns

In [ ]:
mal

## Finally I'm going to calculate the BP-RP errors.
SQL is broken and I can't add this script to get_data.ipynb, so I'm doing it here.

In [ ]:
from astropy.table import Table
gkf = Table.read('../../data/kepler_dr2_1arcsec.fits', format='fits').to_pandas()

In [ ]:
gkf = Table.read('../../data/kepler_dr2_1arcsec.fits', format='fits').to_pandas().rename(columns={
                    'kepid':'KIC'})[['KIC','phot_bp_mean_flux',
                                     'phot_bp_mean_flux_error',
                                     'phot_rp_mean_flux',
                                     'phot_rp_mean_flux_error']]

In [ ]:
ebp = gkf['phot_bp_mean_flux_error'] / (gkf['phot_bp_mean_flux']) * np.log(10)
erp = gkf['phot_rp_mean_flux_error'] / (gkf['phot_rp_mean_flux']) * np.log(10)
gkf['ebprp'] = np.sqrt(ebp**2 + erp**2)
mal = pd.merge(mal, gkf[['KIC','ebprp']], on='KIC', how='left')

In [ ]:
mal.to_csv('../../data/atium.csv')

## Having a look at the data

In [ ]:
len(mal.loc[mal.hrclass == 'MS'])

In [ ]:
mal.loc[mal.hrclass == 'MS']

## By-Eye investigations

I'm going to add flags based on by-eye inspection of the corner plots as well.

`-1 : no data`

`0   : no issues`

`1 : poorly constrained rotational parameters`

`2  : bimodal distributions`

`3   : divergence/poor sampling`

In [ ]:
import corner
choice = input('Are we going to investigate the corners? (y/n) ')

if choice == 'y':
    for idx in range(95):
        kic = mal.loc[idx].KIC
        files = glob.glob('/home/oliver/PhD/mnt/RDS/malatium/peakbag/{}/*chains.csv'.format(str(kic)))

        try:
            
            chains = pd.read_csv(files[0],index_col=0)
        except IndexError:
            print('Star {} has not completed yet | idx {}'.format(str(kic), idx))
            mal.loc[idx, 'flagch'] = -1
            continue
            
        labels=['xsplit','cosi','i','split']
        chain = np.array([chains[label] for label in labels])
        
        corner.corner(chain.T, labels=labels, quantiles=[0.16, 0.5, 0.84]
                      ,show_titles=True)
        plt.show()
        
        print('KIC {}\n\
              0   : no issues \n\
              1 : poorly constrained rotational parameters\n\
              2  : bimodal distributions\n\
              3   : not converged'.format(str(kic)))
        flag = int(input('Flag: '))
        mal.loc[idx, 'chainflag'] = flag
        
else:
    mal['chainflag'] = pd.read_csv('../../data/atium.csv',usecols=['chainflag'])

### Finally I'm going to calculate the BP-RP errors.
SQL is broken and I can't add this script to get_data.ipynb, so I'm doing it here.

In [ ]:
from astropy.table import Table
gkf = Table.read('../../data/kepler_dr2_1arcsec.fits', format='fits').to_pandas()

In [ ]:
gkf = Table.read('../../data/kepler_dr2_1arcsec.fits', format='fits').to_pandas().rename(columns={
                    'kepid':'KIC'})[['KIC','phot_bp_mean_flux',
                                     'phot_bp_mean_flux_error',
                                     'phot_rp_mean_flux',
                                     'phot_rp_mean_flux_error']]

In [ ]:
ebp = gkf['phot_bp_mean_flux_error'] / (gkf['phot_bp_mean_flux']) * np.log(10)
erp = gkf['phot_rp_mean_flux_error'] / (gkf['phot_rp_mean_flux']) * np.log(10)
gkf['ebprp'] = np.sqrt(ebp**2 + erp**2)
mal = pd.merge(mal, gkf[['KIC','ebprp']], on='KIC', how='left')

In [ ]:
mal.to_csv('../../data/atium.csv')

In [ ]:
mal